### basic chat

In [ ]:
%pip install --upgrade --quiet langchain langchain-openai

In [ ]:
from langchain.llms import OpenAI

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"]=userdata.get('OPENAI_KEY')

In [ ]:
llm=OpenAI(temperature=0.6)

In [ ]:
text="What is the capital of India"

print(llm.invoke(text))



The capital of India is New Delhi.


### flan t5

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]=userdata.get('HF_TOKEN')

In [ ]:
from langchain import HuggingFaceHub

llm_huggingface=HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0,"max_length":64})

In [ ]:
output=llm_huggingface.invoke("Can you tell me the capital of Russia")
print(output)

moscow


In [ ]:
output = llm_huggingface.invoke("Can you write a poem about AI")   # with the t5

print(output)

i love the way i look at the world i love the way i feel i love the way i think i feel i love the way i feel i love the way i think i feel i love the way i feel i love the way 


In [ ]:
output = llm.invoke("Can you write a poem about AI")   # with openai

print(output)



Oh AI, the future is here
A world of possibilities, without fear
A creation of man, but not quite the same
A mind of its own, with an endless aim

With algorithms and codes, it learns and grows
A digital brain, that constantly flows
Processing data at an incredible speed
It's a marvel of technology, indeed

It can think, it can reason
A virtual being, with endless vision
It can predict, it can adapt
A machine that outsmarts, without a gap

But with all its power, comes a great debate
Will it one day, surpass our own fate?
Will it take over, and rule the land
Or will it be a helpful, guiding hand?

Some say it's a threat, to humanity
A force to be feared, with no morality
But others see, the potential it holds
To make our lives easier, and break new molds

Oh AI, a creation of our own
A reflection of us, yet still unknown
With endless possibilities, and endless strife
Will it be the savior, or the end of life?

Only time will tell, what the future holds
But for now, let's embrace it, 

### Prompt Templates And LLMChain

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables=['country'],
template="Tell me the capital of this {country}")

prompt_template.format(country="India")

'Tell me the capital of this India'

In [ ]:
from langchain.chains import LLMChain

chain=LLMChain(
    llm=llm,
    prompt=prompt_template
    )
print(chain.invoke("India"))

{'country': 'India', 'text': '\n\nThe capital of India is New Delhi.'}


### Combining Multiple Chains using simple Sequential Chain

In [ ]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(
    llm=llm,
    prompt=capital_template,
    )

famous_template=PromptTemplate(
    input_variables=['capital'],
    template="Suggest me some amazing places to visit in {capital}")


In [ ]:
famous_chain = LLMChain(
    llm=llm,
    prompt=famous_template,
    )

In [ ]:
from langchain.chains import SimpleSequentialChain

chain=SimpleSequentialChain(
    chains=[
        capital_chain,
        famous_chain
        ]
    )

print(chain.invoke("India")["output"])

 Some amazing places to visit in New Delhi are:

1. Red Fort: A UNESCO World Heritage Site, the Red Fort is a must-visit for its stunning architecture and historical significance.

2. Qutub Minar: Another UNESCO World Heritage Site, Qutub Minar is the tallest brick minaret in the world and a popular tourist attraction.

3. India Gate: A war memorial dedicated to Indian soldiers, India Gate is a popular picnic spot and a symbol of national pride.

4. Lotus Temple: This stunning architectural marvel is a Bahá'í House of Worship and welcomes people of all religions to pray or meditate.

5. Jama Masjid: The largest mosque in India, Jama Masjid is a beautiful example of Mughal architecture and a popular place for worship.

6. Humayun's Tomb: Another UNESCO World Heritage Site, Humayun's Tomb is a stunning mausoleum built for the Mughal emperor Humayun.

7. Chandni Chowk: One of the oldest and busiest markets in Delhi, Chandni Chowk is a great place to experience the local culture and try so

### Sequential Chain

In [ ]:
capital_template=PromptTemplate(
    input_variables = ['country'],
    template = "Please tell me the capital of the {country}",
    )

capital_chain= LLMChain(
    llm=llm,
    prompt=capital_template,
    output_key="capital",
    )

In [ ]:
famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")

famous_chain = LLMChain(
    llm=llm,
    prompt=famous_template,
    output_key="places",
    )

In [ ]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains=[capital_chain,famous_chain],
    input_variables=['country'],
    output_variables=['capital'],
    )

In [ ]:
chain.invoke({'country':"India"})

{'country': 'India', 'capital': '\n\nThe capital of India is New Delhi.'}

### Chatmodels With ChatOpenAI

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
from langchain.schema import HumanMessage,SystemMessage,AIMessage

In [ ]:
chatllm = ChatOpenAI(temperature=0.6)

In [ ]:
out = chatllm.invoke([
  SystemMessage(content="Yor are a comedian AI assitant"),
  HumanMessage(content="Please provide some comedy punchlines on AI")
  ])
out

AIMessage(content='1. "Why did the AI go to therapy? It had too many unresolved bugs in its code!"\n\n2. "What do you call an AI that tells jokes? A witty algorithm!"\n\n3. "Why did the AI get a job at the bakery? It kneaded the dough!"\n\n4. "Why did the AI always win at poker? It had a perfect poker face...because it didn\'t have a face!"\n\n5. "What\'s an AI\'s favorite type of music? Algo-rhythms!"\n\n6. "Why don\'t AI robots ever get married? They prefer to stay single and avoid all the debugging!"\n\n7. "Why did the AI refuse to play hide-and-seek? It always found you in nanoseconds!"\n\n8. "Why did the AI join a band? It wanted to be the master of all the keyboards!"\n\n9. "What did the AI say to the computer programmer? \'You byte off more than you can chew!\'"\n\n10. "Why did the AI become a stand-up comedian? Because it found the perfect way to process humor: through its circuits!"')

### Prompt Template + LLM +Output Parsers

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [ ]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [ ]:
template= "Your are a helpful assistant. When the use given any input , you should generate 5 words synonyms in a comma seperated list"
human_template= "{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

In [ ]:
chain = chatprompt | chatllm | Commaseperatedoutput()

In [ ]:
chain.invoke({"text":"intelligent"})

['smart', ' clever', ' brilliant', ' knowledgeable', ' astute']

Thanks